In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 
from sklearn.metrics import f1_score, precision_recall_curve

In [3]:

def F1_validate(fn):  

    fp = open(fn)
    lines = fp.readlines() 
    lines = [x.strip() for x in lines]
    fp.close()
    #print(lines[:10])
    
    y_true = []
    y_pred = [] 
    y_prob = [] 
    
    covid_ids = [] 
    non_covid_ids = [] 

    for l in lines: 
        l_splits = l.split(',') 
        #print(l_splits)
        
        #covid class label is 0 
        y_true.append(int(l_splits[1]))  
        pred = int(l_splits[2])
        y_pred.append(pred)    
        y_prob.append(float(l_splits[3]))
        
        if pred==0:
            covid_ids.append(l_splits[0])
        else: 
            non_covid_ids.append(l_splits[0])

    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)


    #F1 = f1_score(y_true, y_pred, average='macro')
    #print(F1)

    F1s = [] 
    threshs = [] 
    F1Max = 0
    threshMax = 0 
    for thresh in np.arange(0.2,0.9,0.01): 
        y_pred2 = [] 
        for p in y_prob:
            pred2 = p > thresh
            y_pred2.append(pred2)
            
        y_pred2 = np.asarray(y_pred2)
        F1 = f1_score(y_true, y_pred2, average='macro')
        F1s.append(F1) 
        threshs.append(thresh)
        
        if F1>F1Max:
            F1Max = F1
            threshMax = thresh 
        
        
        print("thresh = {}, F1 = {}".format(thresh,F1))
        
    return F1s, threshs, F1Max, threshMax 


In [4]:
fn1 = 'BERT-validate-results-split1-203_96.849593_90.702479_checkpoint.pth.tar.txt'
F1s_1,threshs_1,F1Max_1,threshMax_1 = F1_validate(fn1)
print("F1-max = {} at threshold = {}".format(F1Max_1,threshMax_1))


thresh = 0.2, F1 = 0.8821993583360992
thresh = 0.21000000000000002, F1 = 0.8844337536921416
thresh = 0.22000000000000003, F1 = 0.8866635739059359
thresh = 0.23000000000000004, F1 = 0.8911097680943327
thresh = 0.24000000000000005, F1 = 0.8933262805254343
thresh = 0.25000000000000006, F1 = 0.8933262805254343
thresh = 0.26000000000000006, F1 = 0.8977464788732394
thresh = 0.2700000000000001, F1 = 0.8977464788732394
thresh = 0.2800000000000001, F1 = 0.9021500272489145
thresh = 0.2900000000000001, F1 = 0.9021500272489145
thresh = 0.3000000000000001, F1 = 0.9043457256417576
thresh = 0.3100000000000001, F1 = 0.9043457256417576
thresh = 0.3200000000000001, F1 = 0.9022891650867214
thresh = 0.3300000000000001, F1 = 0.9022891650867214
thresh = 0.34000000000000014, F1 = 0.9024244867906668
thresh = 0.35000000000000014, F1 = 0.9025560224089636
thresh = 0.36000000000000015, F1 = 0.9025560224089636
thresh = 0.37000000000000016, F1 = 0.9005034312932192
thresh = 0.38000000000000017, F1 = 0.90050343129321

In [5]:
fn2 = 'BERT-feature-validate-results-split1-030_95.331325_90.495868_L4_checkpoint.pth.tar.txt'
fn3 = 'BERT-feature-validate-results-split1-030_95.362903_90.082645_L16_checkpoint.pth.tar.txt'
#fn2 = 'BERT-feature-validate-results-split1-086_93.805804_90.495868_checkpoint.pth.tar.txt'
F1s_2,threshs_2,F1Max_2,threshMax_2 = F1_validate(fn2)
print("F1-max = {} at threshold = {}".format(F1Max_2,threshMax_2))

F1s_3,threshs_3,F1Max_3,threshMax_3 = F1_validate(fn3)
print("F1-max = {} at threshold = {}".format(F1Max_3,threshMax_3))


thresh = 0.2, F1 = 0.8827472900176628
thresh = 0.21000000000000002, F1 = 0.8849647887323944
thresh = 0.22000000000000003, F1 = 0.8849647887323944
thresh = 0.23000000000000004, F1 = 0.8829205645498247
thresh = 0.24000000000000005, F1 = 0.8851326406835083
thresh = 0.25000000000000006, F1 = 0.8851326406835083
thresh = 0.26000000000000006, F1 = 0.8851326406835083
thresh = 0.2700000000000001, F1 = 0.8873405213114034
thresh = 0.2800000000000001, F1 = 0.8895442735762937
thresh = 0.2900000000000001, F1 = 0.8917439642128806
thresh = 0.3000000000000001, F1 = 0.8961314255431903
thresh = 0.3100000000000001, F1 = 0.8983193277310925
thresh = 0.3200000000000001, F1 = 0.8983193277310925
thresh = 0.3300000000000001, F1 = 0.8983193277310925
thresh = 0.34000000000000014, F1 = 0.9048605013825368
thresh = 0.35000000000000014, F1 = 0.9048605013825368
thresh = 0.36000000000000015, F1 = 0.9028078508067332
thresh = 0.37000000000000016, F1 = 0.9028078508067332
thresh = 0.38000000000000017, F1 = 0.90498004057326

In [6]:
def F1_validate_combo(fn1,fn2,fn3):  

    fp = open(fn1)
    lines = fp.readlines() 
    lines = [x.strip() for x in lines]
    fp.close()
    lines.sort(key=lambda x:x.split(',')[0])
    #print(lines[:10])    

    fp = open(fn2)
    lines2 = fp.readlines() 
    lines2 = [x.strip() for x in lines2]
    fp.close()
    lines2.sort(key=lambda x:x.split(',')[0])
    #print(lines2[:10])
    
    fp = open(fn3)
    lines3 = fp.readlines() 
    lines3 = [x.strip() for x in lines3]
    fp.close()
    lines3.sort(key=lambda x:x.split(',')[0])
    #print(lines3[:10])

    
    y_true = []
    y_prob = [] 
    y_prob2 = [] 
    y_prob3 = [] 
    
    covid_ids = [] 
    non_covid_ids = [] 

    for ind in range(len(lines)):
        
        l = lines[ind]        
        l_splits = l.split(',') 

        l2 = lines2[ind]        
        l2_splits = l2.split(',') 

        l3 = lines3[ind]        
        l3_splits = l3.split(',') 

        assert(l_splits[0]==l2_splits[0] and l2_splits[0]==l3_splits[0]) 
        assert(l_splits[1]==l2_splits[1] and l2_splits[1]==l3_splits[1])

        #print(l_splits)
        
        #covid class label is 0 
        y_true.append(int(l_splits[1]))  
        y_prob.append(float(l_splits[3]))
        y_prob2.append(float(l2_splits[3]))
        y_prob3.append(float(l3_splits[3]))

    y_true = np.asarray(y_true)

    F1s = [] 
    threshs = [] 
    F1Max = 0
    threshMax = 0 
    for thresh in np.arange(0.2,0.9,0.01): 
        y_pred = [] 
        for ind in range(len(y_prob)): 
            
            p = (y_prob[ind]+y_prob2[ind]+y_prob3[ind])/3
            
            pred = p > thresh
            y_pred.append(pred)
            
        y_pred = np.asarray(y_pred)
        F1 = f1_score(y_true, y_pred, average='macro')
        F1s.append(F1) 
        threshs.append(thresh)
        
        if F1>F1Max:
            F1Max = F1
            threshMax = thresh 
        
        
        print("thresh = {}, F1 = {}".format(thresh,F1))
        
    return F1s, threshs, F1Max, threshMax 


In [7]:
F1_123,threshs_123,F1Max_123,threshMax_123 = F1_validate_combo(fn1,fn2,fn3) 
print("F1-max = {} at threshold = {}".format(F1Max_123,threshMax_123))


thresh = 0.2, F1 = 0.8777165602131138
thresh = 0.21000000000000002, F1 = 0.8799603174603176
thresh = 0.22000000000000003, F1 = 0.8803418803418803
thresh = 0.23000000000000004, F1 = 0.8803418803418803
thresh = 0.24000000000000005, F1 = 0.8825693577487902
thresh = 0.25000000000000006, F1 = 0.8847923829674691
thresh = 0.26000000000000006, F1 = 0.8870110249261114
thresh = 0.2700000000000001, F1 = 0.8870110249261114
thresh = 0.2800000000000001, F1 = 0.8870110249261114
thresh = 0.2900000000000001, F1 = 0.8871779985661947
thresh = 0.3000000000000001, F1 = 0.8871779985661947
thresh = 0.3100000000000001, F1 = 0.8893869873248599
thresh = 0.3200000000000001, F1 = 0.8915918223939918
thresh = 0.3300000000000001, F1 = 0.8915918223939918
thresh = 0.34000000000000014, F1 = 0.8961314255431903
thresh = 0.35000000000000014, F1 = 0.8961314255431903
thresh = 0.36000000000000015, F1 = 0.8961314255431903
thresh = 0.37000000000000016, F1 = 0.8940826330532213
thresh = 0.38000000000000017, F1 = 0.89408263305322

In [8]:
def predict_test(fn1,thresh):
    
    fp = open(fn1)
    lines = fp.readlines() 
    fp.close()
    
    lines = [x.strip() for x in lines]
    lines = [x.strip() for x in lines if not 'part_2_test_set_ECCV_22' in x]
    
    lines.sort(key=lambda x: x.split(',')[0].split('/')[-1])
    #lines.sort(key=lambda x: int(x.split(',')[0].split('/')[-1].split('_')[-1]))
    print(lines[:10])
    
    ids = [] 
    preds = [] 
    probs = [] 
    
    for ind in range(len(lines)):
        
        l = lines[ind]        
        l_splits = l.split(',') 

        ct_scan_id = l_splits[0].split('/')[-1]
        ids.append(ct_scan_id)
        
        y_prob = float(l_splits[3])
        
        pred = y_prob>thresh 
        preds.append(pred) 
        probs.append(y_prob)
        
            
    return ids, preds, probs 


In [11]:
def predict_test_combo(fn1,fn2,fn3,thresh):
    
    fp = open(fn1)
    lines = fp.readlines() 
    fp.close()
    
    lines = [x.strip() for x in lines]
    lines = [x.strip() for x in lines if not 'part_2_test_set_ECCV_22' in x]
    lines.sort(key=lambda x: x.split(',')[0].split('/')[-1])
    
    #lines.sort(key=lambda x: int(x.split(',')[0].split('/')[-1].split('_')[-1]))    
    #print(lines[:10])
    
    fp = open(fn2)
    lines2 = fp.readlines() 
    fp.close()
        
    lines2 = [x.strip() for x in lines2]
    lines2 = [x.strip() for x in lines2 if not 'part_2_test_set_ECCV_22' in x]
    lines2.sort(key=lambda x: x.split(',')[0].split('/')[-1])

    fp = open(fn3)
    lines3 = fp.readlines() 
    fp.close()
        
    lines3 = [x.strip() for x in lines3]
    lines3 = [x.strip() for x in lines3 if not 'part_2_test_set_ECCV_22' in x]
    lines3.sort(key=lambda x: x.split(',')[0].split('/')[-1])

    #lines2.sort(key=lambda x: int(x.split(',')[0].split('/')[-1].split('_')[-1]))
    #print(lines2[:10])

    ids = [] 
    preds = [] 
    probs = [] 

    for ind in range(len(lines)):
        
        l = lines[ind]        
        l_splits = l.split(',') 

        l2 = lines2[ind]        
        l2_splits = l2.split(',') 
        
        l3 = lines3[ind]        
        l3_splits = l3.split(',') 
        
        assert(l_splits[0]==l2_splits[0] and l2_splits[0]==l3_splits[0])        
        assert(l_splits[1]==l2_splits[1] and l2_splits[1]==l3_splits[1])        
        
        ct_scan_id = l_splits[0].split('/')[-1]
        ids.append(ct_scan_id)
                
        y_prob = float(l_splits[3])
        y_prob2 = float(l2_splits[3])
        y_prob3 = float(l3_splits[3])
        
        y_prob_combo = (y_prob+y_prob2+y_prob3)/3.0
        
        pred = y_prob_combo > thresh 
        
        preds.append(pred) 
        probs.append(y_prob_combo)
        
            
    return ids, preds, probs 


In [11]:
'''
fn1 = 'BERT-validate-results-split1-203_96.849593_90.702479_checkpoint.pth.tar.txt'
fn2 = 'BERT-feature-validate-results-split1-030_95.362903_90.082645_L16_checkpoint.pth.tar.txt'


ids1, preds1, probs1 = predict_test(fn1,threshMax_1)
ids2, preds2, probs2 = predict_test(fn2,threshMax_2)
ids12, preds12, probs12 = predict_test_combo(fn1,fn2,threshMax_12)

ids3, preds3, probs3 = predict_test(fn1,0.5)
ids4, preds4, probs4 = predict_test(fn2,0.5)

preds = [preds1,preds2,preds12,preds3,preds4]
ids =[ids1,ids2,ids12,ids3,ids4]

for ind in range(5):
    
    covid_fn = 'submission/covid_{}.csv'.format(ind)
    non_covid_fn = 'submission/non-covid_{}.csv'.format(ind)
    fp1 = open(covid_fn,'w')
    fp2 = open(non_covid_fn,'w')
    
    for j in range(len(preds[ind])):
        pred = preds[ind][j]
        id = ids[ind][j]        
        if pred:
            fp2.write('{}\n'.format(id))
        else:
            fp1.write('{}\n'.format(id))
    
    fp1.close()
    fp2.close()
'''


"\nfn1 = 'BERT-validate-results-split1-203_96.849593_90.702479_checkpoint.pth.tar.txt'\nfn2 = 'BERT-feature-validate-results-split1-030_95.362903_90.082645_L16_checkpoint.pth.tar.txt'\n\n\nids1, preds1, probs1 = predict_test(fn1,threshMax_1)\nids2, preds2, probs2 = predict_test(fn2,threshMax_2)\nids12, preds12, probs12 = predict_test_combo(fn1,fn2,threshMax_12)\n\nids3, preds3, probs3 = predict_test(fn1,0.5)\nids4, preds4, probs4 = predict_test(fn2,0.5)\n\npreds = [preds1,preds2,preds12,preds3,preds4]\nids =[ids1,ids2,ids12,ids3,ids4]\n\nfor ind in range(5):\n    \n    covid_fn = 'submission/covid_{}.csv'.format(ind)\n    non_covid_fn = 'submission/non-covid_{}.csv'.format(ind)\n    fp1 = open(covid_fn,'w')\n    fp2 = open(non_covid_fn,'w')\n    \n    for j in range(len(preds[ind])):\n        pred = preds[ind][j]\n        id = ids[ind][j]        \n        if pred:\n            fp2.write('{}\n'.format(id))\n        else:\n            fp1.write('{}\n'.format(id))\n    \n    fp1.close()\n

In [12]:
'''
for ind in range(len(ids1)):
    
    id1 = ids1[ind]
    id2 = ids2[ind]
    id3 = ids12[ind]
    
    assert(id1==id2 and id2==id3)

    if preds1[ind] == preds2[ind] and preds2[ind] == preds12[ind]:
        print("id = {}, all three predctions are the same = {}".format(id1,preds1[ind]))
    else: 
        print("predictions = {}, {}, {}, probs = {}, {}, {}".format(preds1[ind], preds2[ind], preds12[ind],probs1[ind],probs2[ind],probs12[ind]))
'''

'\nfor ind in range(len(ids1)):\n    \n    id1 = ids1[ind]\n    id2 = ids2[ind]\n    id3 = ids12[ind]\n    \n    assert(id1==id2 and id2==id3)\n\n    if preds1[ind] == preds2[ind] and preds2[ind] == preds12[ind]:\n        print("id = {}, all three predctions are the same = {}".format(id1,preds1[ind]))\n    else: \n        print("predictions = {}, {}, {}, probs = {}, {}, {}".format(preds1[ind], preds2[ind], preds12[ind],probs1[ind],probs2[ind],probs12[ind]))\n'

In [19]:
fn1 = 'BERT-test-results-split1-203_96.849593_90.702479_checkpoint.pth.tar.txt'
fn2 = 'BERT-feature-test-results-split1-030_95.331325_90.495868_L4_checkpoint.pth.tar.txt'
fn3 = 'BERT-feature-test-results-split1-030_95.362903_90.082645_L16_checkpoint.pth.tar.txt'
#fn2 = 'BERT-feature-test-results-split1-086_93.805804_90.495868_checkpoint.pth.tar.txt'

ids1, preds1, probs1 = predict_test(fn1,threshMax_1)
ids2, preds2, probs2 = predict_test(fn2,threshMax_2)
ids12, preds12, probs12 = predict_test_combo(fn1,fn2,fn3,threshMax_123)

ids3, preds3, probs3 = predict_test(fn1,0.5)
ids4, preds4, probs4 = predict_test(fn2,0.5)

preds = [preds1,preds2,preds12,preds3,preds4]
ids =[ids1,ids2,ids12,ids3,ids4]

for ind in range(5):

    covid_ids = [] 
    non_covid_ids = [] 
    for j in range(len(preds[ind])):
        pred = preds[ind][j]
        id = ids[ind][j].split('/')[-1]        
        if pred:
            non_covid_ids.append(id)
        else:
            covid_ids.append(id)

    covid_ids.sort(key=lambda x: int(x.split('_')[-1]))
    non_covid_ids.sort(key=lambda x: int(x.split('_')[-1]))
            
    covid_fn = 'submission/covid_{}.csv'.format(ind+1)
    fp = open(covid_fn,'w')
    for id in covid_ids:
        fp.write('{}\n'.format(id))            
    fp.close()
            
    non_covid_fn = 'submission/non-covid_{}.csv'.format(ind+1)
    fp = open(non_covid_fn,'w')
    for id in non_covid_ids:
        fp.write('{}\n'.format(id))            
    fp.close()
    

['test/part1_0_test_set_ECCV/ct_scan_0,0,0,0.014', 'test/part1_0_test_set_ECCV/ct_scan_1,0,1,0.999', 'test/part1_0_test_set_ECCV/ct_scan_10,0,1,0.999', 'test/part1_0_test_set_ECCV/ct_scan_100,0,1,0.999', 'test/part1_0_1_test_set_ECCV/ct_scan_1000,0,1,0.959', 'test/part1_0_1_test_set_ECCV/ct_scan_1001,0,1,0.948', 'test/part1_0_1_test_set_ECCV/ct_scan_1002,0,1,0.972', 'test/part1_0_1_test_set_ECCV/ct_scan_1003,0,1,0.978', 'test/part1_0_1_test_set_ECCV/ct_scan_1004,0,1,1.000', 'test/part1_0_1_test_set_ECCV/ct_scan_1005,0,1,0.995']
['test/part1_0_test_set_ECCV/ct_scan_0,0,0,0.010', 'test/part1_0_test_set_ECCV/ct_scan_1,0,1,0.999', 'test/part1_0_test_set_ECCV/ct_scan_10,0,1,0.944', 'test/part1_0_test_set_ECCV/ct_scan_100,0,1,0.997', 'test/part1_0_1_test_set_ECCV/ct_scan_1000,0,1,0.926', 'test/part1_0_1_test_set_ECCV/ct_scan_1001,0,1,0.940', 'test/part1_0_1_test_set_ECCV/ct_scan_1002,0,1,0.981', 'test/part1_0_1_test_set_ECCV/ct_scan_1003,0,1,0.864', 'test/part1_0_1_test_set_ECCV/ct_scan_1004

In [13]:
def predict_test_part2(fn1,thresh):
    
    fp = open(fn1)
    lines = fp.readlines() 
    fp.close()
    
    lines = [x.strip() for x in lines]
    lines = [x.strip() for x in lines if 'part_2_test_set_ECCV_22' in x]
    
    lines.sort(key=lambda x: x.split(',')[0].split('/')[-1])
    #lines.sort(key=lambda x: int(x.split(',')[0].split('/')[-1].split('_')[-1]))
    print(lines[:10])
    
    ids = [] 
    preds = [] 
    probs = [] 
    
    for ind in range(len(lines)):
        
        l = lines[ind]        
        l_splits = l.split(',') 

        ct_scan_id = l_splits[0].split('/')[-1]
        ids.append(ct_scan_id)
        
        y_prob = float(l_splits[3])
        
        pred = y_prob>thresh 
        preds.append(pred) 
        probs.append(y_prob)
        
            
    return ids, preds, probs 


In [17]:
def predict_test_combo_part2(fn1,fn2,fn3,thresh):
    
    fp = open(fn1)
    lines = fp.readlines() 
    fp.close()
    
    lines = [x.strip() for x in lines]
    lines = [x.strip() for x in lines if 'part_2_test_set_ECCV_22' in x]
    lines.sort(key=lambda x: x.split(',')[0].split('/')[-1])
    
    #lines.sort(key=lambda x: int(x.split(',')[0].split('/')[-1].split('_')[-1]))    
    #print(lines[:10])
    
    fp = open(fn2)
    lines2 = fp.readlines() 
    fp.close()
        
    lines2 = [x.strip() for x in lines2]
    lines2 = [x.strip() for x in lines2 if 'part_2_test_set_ECCV_22' in x]
    lines2.sort(key=lambda x: x.split(',')[0].split('/')[-1])

    fp = open(fn3)
    lines3 = fp.readlines() 
    fp.close()
        
    lines3 = [x.strip() for x in lines3]
    lines3 = [x.strip() for x in lines3 if 'part_2_test_set_ECCV_22' in x]
    lines3.sort(key=lambda x: x.split(',')[0].split('/')[-1])
    
    #lines2.sort(key=lambda x: int(x.split(',')[0].split('/')[-1].split('_')[-1]))
    #print(lines2[:10])

    ids = [] 
    preds = [] 
    probs = [] 

    for ind in range(len(lines)):
        
        l = lines[ind]        
        l_splits = l.split(',') 

        l2 = lines2[ind]        
        l2_splits = l2.split(',') 
        
        l3 = lines3[ind]        
        l3_splits = l3.split(',') 
        
        assert(l_splits[0]==l2_splits[0] and l2_splits[0]==l3_splits[0]) 
        assert(l_splits[1]==l2_splits[1] and l2_splits[1]==l3_splits[1])        
        
        ct_scan_id = l_splits[0].split('/')[-1]
        ids.append(ct_scan_id)
                
        y_prob = float(l_splits[3])
        y_prob2 = float(l2_splits[3])
        y_prob3 = float(l3_splits[3])
        
        y_prob_combo = (y_prob+y_prob2+y_prob3)/3.0
        
        pred = y_prob_combo > thresh 
        
        preds.append(pred) 
        probs.append(y_prob_combo)
        
            
    return ids, preds, probs 


In [18]:
ids1, preds1, probs1 = predict_test_part2(fn1,threshMax_1)
ids2, preds2, probs2 = predict_test_part2(fn2,threshMax_2)
ids12, preds12, probs12 = predict_test_combo_part2(fn1,fn2,fn3,threshMax_123)

ids3, preds3, probs3 = predict_test_part2(fn1,0.5)
ids4, preds4, probs4 = predict_test_part2(fn2,0.5)

preds = [preds1,preds2,preds12,preds3,preds4]
ids =[ids1,ids2,ids12,ids3,ids4]

for ind in range(5):
    
    count_covid = 0 
    count_non_covid = 0 
    '''
    covid_fn = 'submission/covid_part2_{}.csv'.format(ind+1)
    non_covid_fn = 'submission/non-covid_part2_{}.csv'.format(ind+1)
    fp1 = open(covid_fn,'w')
    fp2 = open(non_covid_fn,'w')
    '''
    for j in range(len(preds[ind])):
        pred = preds[ind][j]
        id = ids[ind][j]        
        if pred:
            #fp2.write('{}\n'.format(id))
            count_non_covid += 1 
        else:
            #fp1.write('{}\n'.format(id))
            count_covid +=1 
    
    print("ind = {}, acc = {}".format(ind+1,count_covid/(count_covid+count_non_covid)))
    
    #fp1.close()
    #fp2.close()

['test/part_2_test_set_ECCV_22/test_ct_scan_0,0,0,0.012', 'test/part_2_test_set_ECCV_22/test_ct_scan_1,0,0,0.005', 'test/part_2_test_set_ECCV_22/test_ct_scan_10,0,1,0.903', 'test/part_2_test_set_ECCV_22/test_ct_scan_100,0,1,0.987', 'test/part_2_test_set_ECCV_22/test_ct_scan_101,0,0,0.033', 'test/part_2_test_set_ECCV_22/test_ct_scan_102,0,0,0.024', 'test/part_2_test_set_ECCV_22/test_ct_scan_103,0,0,0.004', 'test/part_2_test_set_ECCV_22/test_ct_scan_104,0,0,0.035', 'test/part_2_test_set_ECCV_22/test_ct_scan_105,0,0,0.175', 'test/part_2_test_set_ECCV_22/test_ct_scan_106,0,0,0.004']
['test/part_2_test_set_ECCV_22/test_ct_scan_0,0,0,0.000', 'test/part_2_test_set_ECCV_22/test_ct_scan_1,0,0,0.000', 'test/part_2_test_set_ECCV_22/test_ct_scan_10,0,0,0.128', 'test/part_2_test_set_ECCV_22/test_ct_scan_100,0,0,0.011', 'test/part_2_test_set_ECCV_22/test_ct_scan_101,0,0,0.004', 'test/part_2_test_set_ECCV_22/test_ct_scan_102,0,0,0.019', 'test/part_2_test_set_ECCV_22/test_ct_scan_103,0,0,0.007', 'test